# ACRIS Real Property Transactions Analysis for Office Apocalypse Algorithm

## Overview
This notebook analyzes NYC ACRIS (Automated City Register Information System) real property transaction data to understand **market dynamics**, **ownership patterns**, and **investment flows** that are crucial indicators of office building market health. ACRIS provides unique insights into property market activity that no other dataset can capture.

## Why ACRIS Data is Critical:
- **Market Sentiment Indicator**: Transaction volume and pricing reflect investor confidence
- **Ownership Stability**: Frequent ownership changes may signal distressed properties
- **Investment Patterns**: Transaction types reveal market dynamics (sales, mortgages, foreclosures)
- **Market Timing**: Transaction dates capture market cycles and trends

## Unique Market Intelligence Only ACRIS Provides:
- **Real Money Flows**: Actual transaction values show true market valuations
- **Ownership Networks**: Corporate ownership patterns reveal institutional vs. individual investment
- **Market Stress Signals**: Foreclosure filings and distressed sales indicate problem areas
- **Investment Velocity**: Transaction frequency shows market liquidity and confidence

## Impact on Office Building Prediction:
- High transaction volume → Active, healthy office market
- Stable ownership → Lower vacancy risk
- Investment grade transactions → Premium office locations
- Distressed sales patterns → Higher vacancy probability

## Dataset Source
- **File**: `ACRIS_-_Real_Property_Legals_20250915.csv`
- **Source**: NYC Department of Finance, City Register
- **Coverage**: All recorded real property transactions in NYC

In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configure plotting
plt.style.use('default')
sns.set_palette("coolwarm")
plt.rcParams['figure.figsize'] = (12, 8)

## 1. Load and Explore ACRIS Transaction Structure

Let's examine the ACRIS dataset to understand how real property transactions reveal market dynamics and investment patterns crucial for office building analysis.

In [ ]:
# Load ACRIS real property transactions dataset
acris_path = r"c:\Users\pcric\Desktop\capstone_project\office_apocalypse_algorithm_project\data\raw\ACRIS_-_Real_Property_Legals_20250915.csv"
print("Loading ACRIS Real Property Transactions dataset...")
df_acris = pd.read_csv(acris_path)

print(f"Dataset shape: {df_acris.shape}")
print(f"Number of property transactions: {len(df_acris):,}")
print(f"Number of features: {len(df_acris.columns)}")
print("\n" + "="*60)
print("ACRIS Transaction Dataset Overview")
print("="*60)

In [ ]:
# Explore basic structure and identify market intelligence
print("Dataset Info:")
df_acris.info()
print("\n" + "="*50)
print("First 5 rows:")
print(df_acris.head())

print("\n" + "="*50)
print("Market Intelligence Indicators Available:")
print("="*50)

# Identify critical columns for market analysis
market_columns = []
for col in df_acris.columns:
    print(f"✓ {col}")
    if any(keyword in col.lower() for keyword in ['document', 'amount', 'sale', 'price', 'date', 'type', 'party', 'address', 'bbl']):
        market_columns.append(col)

print(f"\nColumns critical for market intelligence:")
for col in market_columns:
    print(f"  → {col}")
    
print(f"\nThis dataset enables analysis of:")
print("• Real property transaction patterns and volume")
print("• Market pricing trends and valuations") 
print("• Ownership change frequency and stability")
print("• Investment activity and market confidence")
print("• Distressed property identification")

## 2. ACRIS Market Intelligence Visualizations

These visualizations demonstrate how property transaction data reveals market dynamics and investment patterns critical for office building analysis.

In [ ]:
# ACRIS Market Dynamics and Investment Pattern Analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('ACRIS Transactions: Market Intelligence for Office Building Investment Analysis', fontsize=16, fontweight='bold')

# Document type analysis
doc_type_cols = [col for col in df_acris.columns if any(keyword in col.lower() for keyword in ['document', 'type', 'class'])]
if doc_type_cols:
    doc_type_col = doc_type_cols[0]
    doc_types = df_acris[doc_type_col].value_counts().head(12)
    
    axes[0,0].barh(range(len(doc_types)), doc_types.values, color='darkblue')
    axes[0,0].set_yticks(range(len(doc_types)))
    axes[0,0].set_yticklabels([str(x)[:25] + '...' if len(str(x)) > 25 else str(x) for x in doc_types.index], fontsize=8)
    axes[0,0].set_title('Transaction Types Distribution\n(Market activity patterns)', fontweight='bold')
    axes[0,0].set_xlabel('Number of Transactions')
    axes[0,0].grid(True, alpha=0.3)

# Transaction amount analysis (if available)
amount_cols = [col for col in df_acris.columns if any(keyword in col.lower() for keyword in ['amount', 'price', 'value', 'consideration'])]
if amount_cols:
    amount_col = amount_cols[0]
    # Filter out zero/unrealistic amounts
    valid_amounts = df_acris[df_acris[amount_col] > 0]
    if len(valid_amounts) > 0:
        # Use log scale for better visualization
        log_amounts = np.log10(valid_amounts[amount_col])
        
        axes[0,1].hist(log_amounts, bins=50, alpha=0.7, color='darkred', edgecolor='black')
        axes[0,1].set_title('Transaction Value Distribution (Log Scale)\n(Market pricing intelligence)', fontweight='bold')
        axes[0,1].set_xlabel('Log10(Transaction Amount $)')
        axes[0,1].set_ylabel('Number of Transactions')
        axes[0,1].grid(True, alpha=0.3)

# Geographic distribution by borough (if available)
borough_cols = [col for col in df_acris.columns if 'borough' in col.lower()]
if borough_cols:
    borough_col = borough_cols[0]
    borough_dist = df_acris[borough_col].value_counts()
    
    axes[1,0].pie(borough_dist.values, labels=borough_dist.index, autopct='%1.1f%%', startangle=90)
    axes[1,0].set_title('Transaction Distribution by Borough\n(Investment flow patterns)', fontweight='bold')

# Temporal analysis (if date available)
date_cols = [col for col in df_acris.columns if any(keyword in col.lower() for keyword in ['date', 'recorded', 'filed'])]
if date_cols:
    date_col = date_cols[0]
    # Convert to datetime and extract year/month for trends
    try:
        df_acris[date_col] = pd.to_datetime(df_acris[date_col], errors='coerce')
        df_acris['transaction_year'] = df_acris[date_col].dt.year
        
        # Filter to recent years for trend analysis
        recent_years = df_acris[df_acris['transaction_year'] >= 2020]
        if len(recent_years) > 0:
            yearly_transactions = recent_years['transaction_year'].value_counts().sort_index()
            
            axes[1,1].plot(yearly_transactions.index, yearly_transactions.values, marker='o', linewidth=3, color='purple')
            axes[1,1].set_title('Annual Transaction Volume Trend\n(Market activity timeline)', fontweight='bold')
            axes[1,1].set_xlabel('Year')
            axes[1,1].set_ylabel('Number of Transactions')
            axes[1,1].grid(True, alpha=0.3)
        else:
            axes[1,1].text(0.5, 0.5, 'Temporal analysis requires\nvalid date data', 
                          ha='center', va='center', transform=axes[1,1].transAxes, fontsize=12)
            axes[1,1].set_title('Annual Transaction Volume Trend\n(Market activity timeline)', fontweight='bold')
    except:
        axes[1,1].text(0.5, 0.5, 'Date parsing issues\nrequire data cleaning', 
                      ha='center', va='center', transform=axes[1,1].transAxes, fontsize=12)
        axes[1,1].set_title('Annual Transaction Volume Trend\n(Market activity timeline)', fontweight='bold')

plt.tight_layout()
plt.show()

print("🎯 KEY INSIGHT: ACRIS reveals market intelligence invisible in other datasets:")
print("   • Transaction types show market activity patterns (sales, mortgages, etc.)")
print("   • Property values reveal true market pricing and investment flows")
print("   • Geographic patterns show investment concentration and market hotspots")
print("   • Temporal trends capture market cycles and confidence changes")
print("\n💡 ONLY ACRIS DATA provides real market transaction intelligence!")